In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
from ccfj import GetStationPairs
from geopy.distance import great_circle
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
#import shapefile
import geopandas as gp
import yaml
import math
#from pykml import parser


In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [ ]:
flag_project = 0 # 0--regular; 1--repartrition

In [ ]:
if flag_project == 0:
    with open('a-project.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project/output_FJSJ_16-01/'               # Harmon server
elif flag_project == 1:
    with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
        proj = yaml.load(f.read(), Loader=yaml.FullLoader)
    name_project = proj['name']
    #name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server


In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
file_xlsx = 'structure_model.xlsx'
file_xlsx_fund = 'structure_model_fund.xlsx'

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary


In [ ]:
dir_image = dir_project + info_basic['dir_image']
if file_xlsx == 'structure_model_fund.xlsx':
    dir_image = dir_project + info_basic['dir_image_inv_fund']
else:
    dir_image = dir_project + info_basic['dir_image_inv']
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

In [ ]:

struc = {}
flag = 0
file_struc = dir_inv_dispernet + file_xlsx
df = pd.read_excel(file_struc,sheet_name=None)
for key_subwork in key_subworks:
    struc[key_subwork] = df[key_subwork]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))


In [ ]:

struc_fund = {}
flag = 0
file_struc = dir_inv_dispernet + file_xlsx_fund
df = pd.read_excel(file_struc,sheet_name=None)
for key_subwork in key_subworks:
    struc_fund[key_subwork] = df[key_subwork]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))


In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_all =  stainfo['latitude'].tolist() 
lon_all =  stainfo['longitude'].tolist() 

In [ ]:
key_subworks = info_basic['key_subworks']
lat_this = []
lon_this = []
for key_subwork in key_subworks:
    key = 'R'+key_subwork[0:2]+key_subwork[3:]
    if key in stalist_all:
        lat_this.append(lat_all[stalist_all.index(key)])
        lon_this.append(lon_all[stalist_all.index(key)])
    else:
        key1 = 'R' + str(int(key_subwork[0:2])-1).zfill(2)+key_subwork[3:5]
        key2 = 'R' + str(int(key_subwork[0:2])+1).zfill(2)+key_subwork[3:5]
        indx1 = stalist_all.index(key1)
        indx2 = stalist_all.index(key2)
        lat_this.append((lat_all[indx1]+lat_all[indx2])/2)
        lon_this.append((lon_all[indx1]+lon_all[indx2])/2)

### Plot horizontal of certain depth

In [ ]:
def extract_horizontal(M,N,key_subworks,lat_all,lon_all,d,struc):
    rows = []
    cols = []

    vs_all = np.zeros(len(key_subworks))
    lats_this = np.zeros(len(key_subworks))
    lons_this = np.zeros(len(key_subworks))

    for i in range(len(key_subworks)):
        key_subwork = key_subworks[i]
        key = 'R' +  key_subwork[0:2]+key_subwork[3:5]
        rows.append(int(key_subwork[3:5]))
        cols.append(int(key_subwork[0:2]))
        if key in stalist_all:
            indx = stalist_all.index(key)
            lats_this[i] = lat_all[indx]
            lons_this[i] = lon_all[indx]
            vs_all[i] = struc[key_subwork]['vs'].tolist()[N-1]
        else:
            key1 = 'R' + str(int(key_subwork[0:2])-1).zfill(2)+key_subwork[3:5]
            key2 = 'R' + str(int(key_subwork[0:2])+1).zfill(2)+key_subwork[3:5]
            indx1 = stalist_all.index(key1)
            indx2 = stalist_all.index(key2)
            lats_this[i] = (lat_all[indx1]+lat_all[indx2])/2
            lons_this[i] = (lon_all[indx1]+lon_all[indx2])/2
            vs_all[i] = struc[key_subwork]['vs'].tolist()[N]

    rows_flag = set(np.sort(rows))
    cols_flag = set(np.sort(cols))

    # reshape
    vs = np.zeros((len(rows_flag),len(cols_flag)))
    lats = np.zeros((len(rows_flag),len(cols_flag)))
    lons = np.zeros((len(rows_flag),len(cols_flag)))
    for i in range(len(rows_flag)):
        for j in range(len(cols_flag)):
            indx = i*len(cols_flag)+j
            vs[i,j] = vs_all[indx]
            lats[i,j] = lats_this[indx]
            lons[i,j] = lons_this[indx]

    # 2D interpolation
    #y = np.array(lon_this)
    #x = np.array(lat_this)     
    x = np.array(lon_this)
    y = np.array(lat_this)  
    angle = math.radians(d)
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])
    new_coordinates = np.dot(rotation_matrix, np.vstack((x, y)))
    x_rotated, y_rotated = new_coordinates
    # generate meshigrid
    xx_inter = np.linspace(min(x_rotated),max(x_rotated),M)
    yy_inter = np.linspace(min(y_rotated),max(y_rotated),M)
    x_inter_r , y_inter_r = np.meshgrid(xx_inter,yy_inter)

    angle = math.radians(-d)
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],[np.sin(angle), np.cos(angle)]])
    new_coordinates = np.dot(rotation_matrix, np.vstack((x_inter_r.reshape(-1), y_inter_r.reshape(-1))))
    x_inter, y_inter = new_coordinates

    vs = griddata((x_rotated, y_rotated), vs_all, (xx_inter[None,:], yy_inter[:,None]), method='cubic')


    return vs,y_inter,x_inter,x,y,vs_all

In [ ]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [ ]:
#%%capture
M = 100
dz = 4
N = 50


d = -27.3

for i in range(N):
#for i in range(40,41):
    if i*dz <= 30:
        c_min = 0.3
        c_max = 0.6
    elif i*dz <= 60:
        c_min = 0.5
        c_max = 0.8
    elif i*dz < 120:
        c_min = 0.7
        c_max = 1.1
    else:
        c_min = 0.8
        c_max = 1.3 
    vs,yy,xx,xx_0,yy_0,vs_all = extract_horizontal(M,i,key_subworks,lat_all,lon_all,d,struc)
    vs_fund,yy,xx,xx_0,yy_0,vs_all_fund = extract_horizontal(M,i,key_subworks,lat_all,lon_all,d,struc_fund)
    fig,ax = plt.subplots(2,2,figsize=(20,14))

    ax[0][0].scatter(lon_all, lat_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax[0][0].set_xlabel('Longitude')
    ax[0][0].set_ylabel('Latitude')
    ax[0][0].set_title('Vs at depth '+str((i+1)*dz))
    flag_add = 0.0005
    ax[0][0].set_xlim([min(lon_all)-flag_add, max(lon_all)+flag_add])
    ax[0][0].set_ylim([min(lat_all)-flag_add, max(lat_all)+flag_add])
    #im = ax[0][0].scatter(xx_0,yy_0,c=vs_all, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    im = ax[0][0].scatter(xx_0,yy_0,c=vs_all, cmap='gist_rainbow')
    plt.colorbar(im)
    for i in range(len(faults)):
        ax[0][0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    ax[0][0].set_title('Overtones Depth = '+str((i+1)*dz) + 'm')



    ax[1][0].scatter(lon_all, lat_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax[1][0].set_xlabel('Longitude')
    ax[1][0].set_ylabel('Latitude')
    ax[1][0].set_title('Vs at depth '+str((i+1)*dz))
    flag_add = 0.0005
    ax[1][0].set_xlim([min(lon_all)-flag_add, max(lon_all)+flag_add])
    ax[1][0].set_ylim([min(lat_all)-flag_add, max(lat_all)+flag_add])
    #im = ax[1][0].pcolormesh(xx.reshape(M,M),yy.reshape(M,M),vs,cmap='gist_rainbow')
    #im = ax[1][0].pcolormesh(xx.reshape(M,M),yy.reshape(M,M),vs, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    im = ax[1][0].scatter(xx_0,yy_0,c=vs_all, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    plt.colorbar(im)
    for i in range(len(faults)):
        ax[1][0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    ax[1][0].set_title('Overtones Depth = '+str((i+1)*dz) + 'm')


    ax[0][1].scatter(lon_all, lat_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax[0][1].set_xlabel('Longitude')
    ax[0][1].set_ylabel('Latitude')
    ax[0][1].set_title('Vs at depth '+str((i+1)*dz))
    flag_add = 0.0005
    ax[0][1].set_xlim([min(lon_all)-flag_add, max(lon_all)+flag_add])
    ax[0][1].set_ylim([min(lat_all)-flag_add, max(lat_all)+flag_add])
    #im = ax[0][1].scatter(xx_0,yy_0,c=vs_all_fund, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    im = ax[0][1].scatter(xx_0,yy_0,c=vs_all_fund, cmap='gist_rainbow')
    plt.colorbar(im)
    for i in range(len(faults)):
        ax[0][1].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    ax[0][1].set_title('Fundamental Depth = '+str((i+1)*dz) + 'm')


    ax[1][1].scatter(lon_all, lat_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax[1][1].set_xlabel('Longitude')
    ax[1][1].set_ylabel('Latitude')
    ax[1][1].set_title('Vs at depth '+str((i+1)*dz))
    flag_add = 0.0005
    ax[1][1].set_xlim([min(lon_all)-flag_add, max(lon_all)+flag_add])
    ax[1][1].set_ylim([min(lat_all)-flag_add, max(lat_all)+flag_add])
    #im = ax[1][1].pcolormesh(xx.reshape(M,M),yy.reshape(M,M),vs_fund,cmap='gist_rainbow')
    #im = ax[1][1].pcolormesh(xx.reshape(M,M),yy.reshape(M,M),vs_fund, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    im = ax[1][1].scatter(xx_0,yy_0,c=vs_all_fund, vmin = c_min, vmax=c_max,cmap='gist_rainbow')
    #ax[1].colorbar(im)
    for i in range(len(faults)):
        ax[1][1].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    ax[1][1].set_title('Fundamenal Depth= '+str((i+1)*dz) + 'm')
    plt.colorbar(im)
    #ax.plot(lons_coyote, lats_coyote, 'k')
    filename = dir_image + 'Vs_hori_profile_'+str((i+1)*dz)+'m.png'
    plt.savefig(filename,dpi=100)
    plt.tight_layout()
    plt.close()

### Do clipper

In [ ]:
def extract_verticle(row_tag,N,M,key_subworks,struc):
    key_subworks_lists = []
    rows = []
    cols = []
    for key_subwork in key_subworks:
        row = key_subwork[3:5]
        col = key_subwork[0:2] 
        if row == str(row_tag):
            key_subworks_lists.append(key_subwork)
            rows.append(row)
            cols.append(col)
    
    # 根据行号排序
    rows = np.array(rows)
    cols = np.array(cols)
    indx = np.argsort(cols)
    rows = rows[indx]
    cols = cols[indx]
    key_subworks_lists = np.array(key_subworks_lists)
    key_subworks_lists = key_subworks_lists[indx]

    vs = np.zeros((len(key_subworks_lists),N))
    rs = np.zeros(len(key_subworks_lists))
    
    # 定义起始点的坐标
    key_start = 'R' + cols[0] + rows[0]
    if key_start in stalist_all:
        indx = stalist_all.index(key_start)
        lat_start = lat_all[indx]
        lon_start = lon_all[indx]
    else:
        key1 = 'R' + str(int(cols[0])-1).zfill(2)+rows[0]
        key2 = 'R' + str(int(cols[0])+1).zfill(2)+rows[0]
        indx1 = stalist_all.index(key1)
        indx2 = stalist_all.index(key2)
        lat_start = (lat_all[indx1]+lat_all[indx2])/2
        lon_start = (lon_all[indx1]+lon_all[indx2])/2

    # 算r
    flag_this = 0
    for key_subwork in key_subworks_lists:
        key = 'R' +  key_subwork[0:2]+key_subwork[3:5]
        
        if key in stalist_all:
            indx = stalist_all.index(key)
            rs[flag_this] = great_circle((lat_start,lon_start),(lat_all[indx],lon_all[indx])).m
            vs[flag_this,:] = struc[key_subwork]['vs'].tolist()[0:N]
        else:
            key1 = 'R' + str(int(key_subwork[0:2])-1).zfill(2)+key_subwork[3:5]
            key2 = 'R' + str(int(key_subwork[0:2])+1).zfill(2)+key_subwork[3:5]
            indx1 = stalist_all.index(key1)
            indx2 = stalist_all.index(key2)
            lat_this = (lat_all[indx1]+lat_all[indx2])/2
            lon_this = (lon_all[indx1]+lon_all[indx2])/2
            rs[flag_this] = great_circle((lat_start,lon_start),(lat_this,lon_this)).m
            vs[flag_this,:] = struc[key_subwork]['vs'].tolist()[0:N]
        flag_this += 1
    vs = np.array(vs)

    # 插值
    vs_interp = np.zeros((M*len(rs),N))
    for i in range(N):
        f = interp1d(rs, vs[:,i], kind='cubic')
        vs_interp[:,i] = f(np.linspace(0,rs[-1],M*len(rs)))
    return vs,rs,vs_interp,np.linspace(0,rs[-1],M*len(rs))

In [ ]:
#%%capture
N = 25
M = 1000
c_min = 0.3
c_max = 1.
for i in range(4,17):
    print(i)
#for i in range(5,6):
    plt.figure(figsize=(18,12))
    ax1 = plt.subplot2grid((2,3),(0,0),colspan=2,rowspan=1)
    ax2 = plt.subplot2grid((2,3),(0,2),colspan=1,rowspan=1)
    ax3 = plt.subplot2grid((2,3),(1,0),colspan=2,rowspan=1)
    ax4 = plt.subplot2grid((2,3),(1,2),colspan=1,rowspan=1)

    row_tag = str(i).zfill(2)
    vs_o,rs_o,vs,rs = extract_verticle(row_tag,N,M,key_subworks,struc)
    #fig,ax = plt.subplots(1,1,figsize=(12,6))
    #im = ax1.imshow(vs.T,aspect='auto',cmap='gist_rainbow',vmin=c_min, vmax=c_max,extent=[0,rs[-1],N*dz,0])
    im = ax1.imshow(vs.T,aspect='auto',cmap='gist_rainbow',vmin=c_min, vmax=c_max,extent=[0,rs[-1],N*dz,0])
    # 画vs的等高线
    #cbar = np.linspace(c_min,c_max,20)
    #ax.contourf(xi,yi,zi,15,cmap=plt.cm.jet,vmin=c_min, vmax=c_max)
    ax1.set_xlabel('Distance along the profile (m)')
    ax1.set_ylabel('Depth (m)')
    ax1.set_title('Overtones Vs profile at row '+row_tag)
    #在图中展示colorbar
    cbar = fig.colorbar(im)

    vs_fund_o,rs_o,vs_fund,rs = extract_verticle(row_tag,N,M,key_subworks,struc_fund)
    #fig,ax = plt.subplots(1,1,figsize=(12,6))
    im = ax3.imshow(vs_fund.T,aspect='auto',cmap='gist_rainbow',vmin=c_min, vmax=c_max,extent=[0,rs[-1],N*dz,0])
    #im = ax3.imshow(vs_fund_o.T,aspect='auto',cmap='gist_rainbow',vmin=c_min, vmax=c_max,extent=[0,rs_o[-1],N*dz,0])
    # 画vs的等高线
    #cbar = np.linspace(c_min,c_max,20)
    #ax.contourf(xi,yi,zi,15,cmap=plt.cm.jet,vmin=c_min, vmax=c_max)
    ax3.set_xlabel('Distance along the profile (m)')
    ax3.set_ylabel('Depth (m)')
    ax3.set_title('Fundamental Vs profile at row '+row_tag)
    #在图中展示colorbar
    cbar = fig.colorbar(im)


    lat = []
    lon = []
    for key_subwork in key_subworks:
        key = 'R' +  key_subwork[0:2]+key_subwork[3:5]
        if key_subwork[3:] == row_tag:
            if key in stalist_all:
                indx = stalist_all.index(key)
                lat.append(lat_all[indx])
                lon.append(lon_all[indx])
            else:
                key1 = 'R' + str(int(key_subwork[0:2])-1).zfill(2)+key_subwork[3:5]
                key2 = 'R' + str(int(key_subwork[0:2])+1).zfill(2)+key_subwork[3:5]
                indx1 = stalist_all.index(key1)
                indx2 = stalist_all.index(key2)
                lat_this = (lat_all[indx1]+lat_all[indx2])/2
                lon_this = (lon_all[indx1]+lon_all[indx2])/2
                lat.append(lat_this)
                lon.append(lon_this)

    ax2=plotlib.plot_area(ax2,lon_all,lat_all,lon,lat)
    for i in range(len(faults)):
        ax2.plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    
    ax4=plotlib.plot_area(ax4,lon_all,lat_all,lon,lat)
    for i in range(len(faults)):
        ax4.plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')



    filename = dir_image + 'Vs_profile_row_'+row_tag+'.png'
    plt.tight_layout()
    plt.savefig(filename,dpi=100)